* Azure Machine Learning Studio
* まずはフローを作成する
    * loan_data.csv/loan_status.csvを読み込む
    * Join Dataでマージする
    * Edit Metadata で education/gender,loan_statusをMake categoricalで変換
    * Normalize Data でMinMaxを適用
    * Normalize Data でage列にzscoreを適用し、正規分布に変換
    * Edit Meta Data でloan_ID/due_date/past_due_days/paid_off_timeをFields ⇒ Clear featureとする
    * Edit Meta Data でloan_statusをFields ⇒ Labelと変換する
    * Split Dataでデータ分割
    * Multiclass Decision JungleとSplitDataからTrain Modelへつなげる
    * Train ModelとSplitDataからScore Modelにつなげる
    * Evaluate Modelで評価
* デプロイ
    * Deploy Web Service
* 必要情報の抽出
    * Score ModelからApply SQL Transformationにつなげて、必要な列のみ抽出する
        *select Loan_ID, [Scored Labels] as Prediction from t1;

# Azure Machine Learning

In [1]:
import urllib.parse, http.client, json

service_url = 'ussouthcentral.services.azureml.net'
params = urllib.parse.urlencode({
  'api-version':'2.0',
  'details':True
})

# endpointはMachine Learning Portalの　REQUEST/RESPONSE　をクリックしてドキュメントを見ることができる
endpoint = '/workspaces/xxxx/services/xxxx/execute?%s' % params


# Bearer の後にAPIkeyを書く
headers = {
  'Content-Type':'application/json',
  'Authorization':'Bearer xxxx'
}

body = {
  "Inputs":{
    "input1":{
      "ColumnNames":["Loan_ID", "Principal", "terms", "effective_date", "due_date", "paid_off_time", "past_due_days", "age", "education", "Gender"],
      "Values":[['lfui-gregr-9843543-4qf43wf4', 700, 24, '2020-01-01T00:00:00', '2021-01-01T00:00:00', '', 0, 25, 'college', 'male']]
    }
  },
  "GlobalParameters":{      
  }
}

try:
    conn = http.client.HTTPSConnection(service_url)
    conn.request('POST', endpoint, str(body), headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

{'Results': {'output1': {'type': 'table', 'value': {'ColumnNames': ['Loan_ID', 'Prediction'], 'ColumnTypes': ['String', 'String'], 'Values': [['lfui-gregr-9843543-4qf43wf4', 'PAIDOFF']]}}}}
